# Data loading and process


count the missing values of each dommain features, set the threshold to 0.3 to filter out the features with too many missing values

In [1]:
import pandas as pd
import numpy as np


# data path
file_path = '../data/cumulative_2022_v3_9_domain.csv'

data = pd.read_csv(file_path)

column_to_variable_dict = np.load('../data/column_to_variable_dict.npy', allow_pickle=True).item()
variable_to_column_dict = np.load('../data/variable_to_column_dict.npy', allow_pickle=True).item()

# check the "Year" column's max and min value
print(data['Year'].max())
print(data['Year'].min())

2020.0
1948.0


In [11]:
data.isnull().sum()

Year                                  0
South                              1801
region                             1801
racial_composition_nbhood         59420
racial_composition_gradeSchool    60327
                                  ...  
occupation                        28223
occupation14                      51795
occupation71                      51795
home_ownership                    13672
urbanism                          24972
Length: 116, dtype: int64

In [2]:
# analysis the missing value of each column in differernt period:
# 1. overall missing value and percentage
# 2. missing value and percentage in recent 10 years ("Year" >= 2012)")
# 3. missing value and percentage in recent 20 years ("Year" >= 2002)")
# 4. missing value and percentage in recent 30 years ("Year" >= 1992)")
# 5. missing value and percentage in recent 40 years ("Year" >= 1982)")
# 6. missing value and percentage in recent 50 years ("Year" >= 1972)")
# 7. missing value and percentage in recent 60 years ("Year" >= 1962)")

# save the result in csv file, the first column is the feature name


def missing_value_analysis(data):
    # get the number of missing value of each column
    missing_value_num = data.isnull().sum()
    # get the percentage of missing value of each column
    missing_value_percentage = missing_value_num / len(data)

    missing_value_percentage_10 = data[data['Year'] >= 2012].isnull().sum() / len(data[data['Year'] >= 2012])
    missing_value_percentage_20 = data[data['Year'] >= 2002].isnull().sum() / len(data[data['Year'] >= 2002])
    missing_value_percentage_30 = data[data['Year'] >= 1992].isnull().sum() / len(data[data['Year'] >= 1992])
    missing_value_percentage_40 = data[data['Year'] >= 1982].isnull().sum() / len(data[data['Year'] >= 1982])
    missing_value_percentage_50 = data[data['Year'] >= 1972].isnull().sum() / len(data[data['Year'] >= 1972])
    missing_value_percentage_60 = data[data['Year'] >= 1962].isnull().sum() / len(data[data['Year'] >= 1962])
    missing_value_percentage_70 = data[data['Year'] >= 1952].isnull().sum() / len(data[data['Year'] >= 1952])

    # get the variable name of each column by using the column_to_variable_dict
    # missing_value_num.index = column_to_variable_dict['variable']


    # combine the result
    missing_value = pd.concat([missing_value_num, missing_value_percentage,
                               missing_value_percentage_10, missing_value_percentage_20,
                               missing_value_percentage_30, missing_value_percentage_40,
                               missing_value_percentage_50, missing_value_percentage_60,    missing_value_percentage_70], axis=1)
    missing_value.columns = ['missing_value_num', 'missing_value_percentage',
                                'missing_value_percentage_10(>=2012)', 'missing_value_percentage_20(>=2002)',
                                'missing_value_percentage_30(>=1992)', 'missing_value_percentage_40(>=1982)',
                                'missing_value_percentage_50(>=1972)', 'missing_value_percentage_60(>=1962)', 'missing_value_percentage_60(>=1952)']

    # sort the result by missing value percentage
    missing_value = missing_value.sort_values(by='missing_value_percentage', ascending=False)


    return missing_value

missing_value = missing_value_analysis(data)

# save the result
# massing_value.to_csv('../data/missing_value_analysis.csv')


In [3]:
# add one column to indicate the variable name of each row,using the index of the missing_value as the key

variable_name = [ column_to_variable_dict[var] for var in missing_value.index]
missing_value['variable_name'] = variable_name

In [7]:
# set the filter-out thresholds:
# 1. missing_value_percentage_10(>=2012) < 0.3
# 2. missing_value_percentage_20(>=2002) < 0.4
# 3. missing_value_percentage_30(>=1992) < 0.5

threshold_10 = 0.3
threshold_20 = 0.4
threshold_30 = 0.5


# filter out the features
missing_value_used = missing_value[(
                missing_value['missing_value_percentage_10(>=2012)'] < threshold_10) & 
                                        (missing_value['missing_value_percentage_20(>=2002)'] < threshold_20) &
                                        (missing_value['missing_value_percentage_30(>=1992)'] < threshold_30)]

missing_value_not_used = missing_value[(
                missing_value['missing_value_percentage_10(>=2012)'] >= threshold_10) | 
                                        (missing_value['missing_value_percentage_20(>=2002)'] >= threshold_20) |
                                        (missing_value['missing_value_percentage_30(>=1992)'] >= threshold_30)]

# count the number of features
print('number of features used: ', len(missing_value_used))
print('number of features not used: ', len(missing_value_not_used))



# save the result
# make folder namsed with threshold:

folder_name = '../data/threshold_10_' + str(threshold_10) + '_threshold_20_' + str(threshold_20) + '_threshold_30_' + str(threshold_30)

# make folder if not exist
import os
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# missing_value_used.to_csv(folder_name + '/missing_value_analysis_used.csv')
# missing_value_not_used.to_csv(folder_name + '/missing_value_analysis_not_used.csv')

# save the used features names (row names) and the variable names
used_features = missing_value_used.index.tolist()
with open(folder_name + '/used_features.txt', 'w') as f:
    for item in used_features:
        f.write("%s (%s)\n" % (item, column_to_variable_dict[item]))

# save the not used features names (row names)
not_used_features = missing_value_not_used.index.tolist()
with open(folder_name + '/not_used_features.txt', 'w') as f:
    for item in not_used_features:
        f.write("%s (%s)\n" % (item, column_to_variable_dict[item]))

number of features used:  72
number of features not used:  44


In [8]:
# use the used features to filter out the data

# set the target variable set and index variable set, these variables will not be used for training

target_variable_list = ['Voted','Registered_voted','Voted_party','Vote_Nonvote_Pres']

index_variable_list = ['Year', ]

# check the missing ratio of the target variable
print('missing value of the target variable: ')
print(data[target_variable_list].isnull().sum() / len(data))


missing value of the target variable: 
Voted                0.091551
Registered_voted     0.218061
Voted_party          0.536483
Vote_Nonvote_Pres    0.377067
dtype: float64


In [6]:
target_variable = 'Voted'

'''Voted  {0.0: '0. DK; NA; no Post IW; refused to say if voted;', 1.0: '1. No, did not vote', 2.0: '2. Yes, voted'}'''

# filter out the samples with missing value of the target variable,drop the index
data_new = data[data[target_variable].notnull()]
# filter out the samples with target variable value = 0, count the number of samples whose target variable value = 0, 1 or 2
print('number of samples who not vote : ', len(data_new[data_new[target_variable] == 1]))
print('number of samples who vote : ', len(data_new[data_new[target_variable] == 2]))
print('number of samples who vote case DK : ', len(data_new[data_new[target_variable] == 0]))

data_new = data_new[data_new[target_variable] != 0]
data_new = data_new.reset_index(drop=True)
data_new.shape

number of samples who not vote :  17790
number of samples who vote :  44188
number of samples who vote case DK :  0


(61978, 116)

In [55]:
data_new['Voted'].value_counts()


Voted
2.0    44188
1.0    17790
Name: count, dtype: int64

In [30]:
# go through all used features, check the num of the categories of each feature: if the num of categories > 10, then this feature is a continuous/numerical feature, otherwise, this feature is a categorical feature-> need to do one-hot encoding

numerical_feature_list = []
categorical_feature_list = []

for feature in used_features:

    if feature not in target_variable_list and feature not in index_variable_list:

        if len(data_new[feature].value_counts()) > 10:
            numerical_feature_list.append(feature)
        else:
            categorical_feature_list.append(feature)

print('number of numerical features: ', len(numerical_feature_list))

print('number of categorical features: ', len(categorical_feature_list))

print('numerical features list:',numerical_feature_list)

number of numerical features:  11
number of categorical features:  56
numerical features list: ['therm_Christians', 'therm_Mislims', 'therm_ChrFundament', 'therm_hispanics', 'therm_RepParty', 'therm_DemParty', 'therm_Whites', 'therm_liberals', 'therm_conservatives', 'therm_Blacks', 'Age']


In [60]:

# start from all-clear case:  further filter out the samples with missing value of the used features

data_XY = data_new[numerical_feature_list + categorical_feature_list+[target_variable]]
# data_XY = data_XY[data_XY.notnull().all(axis=1)]
# data_XY = data_XY.reset_index(drop=True)
print(data_XY.shape)

X_continuous = data_XY[numerical_feature_list]
X_categorical = data_XY[categorical_feature_list]
Y_target = data_XY[target_variable]

Y_target.value_counts()

(61978, 68)


Voted
2.0    44188
1.0    17790
Name: count, dtype: int64

Voted
2.0    5087
Name: count, dtype: int64

In [68]:
# only use the continuous features to do logistic regression by sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# impute the missing value of the continuous features by using the mean value of the feature

from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

X_continuous_imp = imp.fit_transform(X_continuous)

X_continuous_transformed = StandardScaler().fit_transform(X_continuous_imp)

X_train, X_test, y_train, y_test = train_test_split(X_continuous_transformed, Y_target, test_size=0.3, random_state=0)

# use the default parameters
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

# get the accuracy, recall, precision, f1 score

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

y_pred = logisticRegr.predict(X_test)

print('accuracy: ', accuracy_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('precision: ', precision_score(y_test, y_pred))
print('f1 score: ', f1_score(y_test, y_pred))


# count the number of predicted samples for each class
print('number of predicted samples for each class: ')
print(pd.Series(y_pred).value_counts())

print('model just predict the majority class: ', pd.Series(y_pred).value_counts()[1] / len(y_pred))

accuracy:  0.7153382811659675
recall:  0.03750234389649353
precision:  0.5555555555555556
f1 score:  0.07026172492534692
number of predicted samples for each class: 
2.0    18234
1.0      360
Name: count, dtype: int64
model just predict the majority class:  0.01936108422071636


In [93]:
# only use the categorical features to do logistic regression by sklearn, for the NaN value, set as a new category, then do one-hot encoding

from sklearn.preprocessing import OneHotEncoder

X_categorical_imp = X_categorical.fillna(-1)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_categorical_imp)
X_categorical_transformed = enc.transform(X_categorical_imp).toarray()

X_train, X_test, y_train, y_test = train_test_split(X_categorical_transformed, Y_target, test_size=0.3, random_state=0)

# use the default parameters
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

# get the accuracy, recall, precision, f1 score

y_pred = logisticRegr.predict(X_test)

print('accuracy: ', accuracy_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('precision: ', precision_score(y_test, y_pred))
print('f1 score: ', f1_score(y_test, y_pred))

# count the number of predicted samples for each class
print('number of predicted samples for each class: ')
print(pd.Series(y_pred).value_counts())



accuracy:  0.8755512530923953
recall:  0.7314832177011064
precision:  0.8155969057077148
f1 score:  0.7712534598655595
number of predicted samples for each class: 
2.0    13811
1.0     4783
Name: count, dtype: int64


/home/zenus/anaconda3/envs/torch_1.13_gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [94]:
# 建立映射
feature_mapping = {}
current_index = 0
for feature_index, categories in enumerate(enc.categories_):
    for category_index, category in enumerate(categories):
        feature_mapping[current_index] = (f'Feature_id {feature_index}', 
         enc.feature_names_in_[feature_index],                             
        f'Category {category_index + 1}', category)
        current_index += 1

# # 打印映射结果
# print("Feature Mapping:")
# for k, v in feature_mapping.items():
#     print(f"Encoded feature {k}: Original {v}")

# identify the top 10 features that have the largest absolute value of the coefficient,

top_10_index = np.argsort(np.abs(logisticRegr.coef_[0]))[-10:]

# print the top 10 features
print('top 10 features: ')
for index in top_10_index:
    print(feature_mapping[index])

top 10 features: 
('Feature_id 7', 'sex_orientation', 'Category 1', -1.0)
('Feature_id 44', 'Interest_elections', 'Category 4', 3.0)
('Feature_id 25', 'VCF9028', 'Category 4', 3.0)
('Feature_id 28', 'Pre_election_inten_vote', 'Category 4', 3.0)
('Feature_id 21', 'church_attendance', 'Category 1', -1.0)
('Feature_id 18', 'volunteer', 'Category 1', -1.0)
('Feature_id 28', 'Pre_election_inten_vote', 'Category 5', 4.0)
('Feature_id 12', 'VCF9022', 'Category 3', 5.0)
('Feature_id 12', 'VCF9022', 'Category 2', 1.0)
('Feature_id 12', 'VCF9022', 'Category 1', -1.0)


In [78]:
# concatenate the continuous features and categorical features, then do logistic regression

X_continuous_categorical = np.concatenate((X_continuous_transformed, X_categorical_transformed), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_continuous_categorical, Y_target, test_size=0.3, random_state=0)

# use the default parameters
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

# get the accuracy, recall, precision, f1 score

y_pred = logisticRegr.predict(X_test)

print('accuracy: ', accuracy_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('precision: ', precision_score(y_test, y_pred))
print('f1 score: ', f1_score(y_test, y_pred))

# check the top 10 features with the highest absolute value of the coefficient

feature_importance = pd.DataFrame({'feature': numerical_feature_list + enc.get_feature_names().tolist(), 'importance': logisticRegr.coef_[0]})

accuracy:  0.8801226202000645
recall:  0.7419838739921245
precision:  0.8226611226611227
f1 score:  0.7802425317953268


/home/zenus/anaconda3/envs/torch_1.13_gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [80]:
logisticRegr.coef_[0]

array([-3.36331930e-02,  3.57453106e-02, -6.44025252e-02, -2.32044616e-02,
       -1.57759918e-02,  1.24256973e-01, -6.48274198e-02, -3.76747785e-02,
        1.11533412e-02,  4.39356074e-02,  4.74777188e-01,  5.92446159e-01,
        1.20615192e-01, -1.62057963e-01, -8.29689575e-02, -2.28963544e-01,
        3.84357878e-01,  1.30349627e-01,  3.02776980e-01, -9.92210552e-02,
       -2.12654556e-02, -4.78535804e-01,  8.39650357e-02, -1.31819897e-01,
        8.12838989e-02,  2.11760017e-01,  7.01381181e-01,  1.14074207e-01,
        8.49576313e-02,  2.69002593e-01,  7.74886118e-01, -3.96971623e-01,
        2.15571722e-01, -2.12243657e-02, -1.04227420e-01, -5.45117870e-01,
        3.80754331e-01, -1.47395657e-01,  1.29110710e-01,  1.46865551e-01,
        5.03817368e-01, -6.30869728e-01,  1.75694613e-01,  2.65303931e-01,
        1.93827085e-01,  2.08633440e-01,  2.55445090e-01,  7.09534804e-01,
       -7.39535251e-02, -3.69983507e-02, -3.25559751e-02, -9.79925218e-02,
       -2.47757618e-01,  

In [77]:
# try the decision tree model

from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X_continuous_categorical, Y_target, test_size=0.3, random_state=0)

# use the default parameters
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# get the accuracy, recall, precision, f1 score

y_pred = clf.predict(X_test)

print('accuracy: ', accuracy_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('precision: ', precision_score(y_test, y_pred))
print('f1 score: ', f1_score(y_test, y_pred))


accuracy:  0.8312358825427557
recall:  0.7108569285580348
precision:  0.7037312047521812
f1 score:  0.7072761194029851


In [47]:
X.columns[:-1]

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4'], dtype='object')

In [43]:
X_train_transformed[:,-3]

array([0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])